In [1]:
import pandas as pd
import numpy as np
from pandas import read_csv
import csv
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn import manifold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR, SVC
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from scipy.spatial.distance import cdist
from sklearn.preprocessing import Imputer
from fancyimpute import SoftImpute
from sklearn.metrics import roc_curve, auc, silhouette_score
from sklearn.metrics import roc_auc_score
import re
from scipy import stats
import glmnet_python
from glmnet import glmnet; from glmnetPlot import glmnetPlot 
from glmnetPrint import glmnetPrint; from glmnetCoef import glmnetCoef; from glmnetPredict import glmnetPredict
from cvglmnet import cvglmnet; from cvglmnetCoef import cvglmnetCoef
from cvglmnetPlot import cvglmnetPlot; from cvglmnetPredict import cvglmnetPredict
import math
import statsmodels.api as sm

Using TensorFlow backend.
/home/ananth/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/ananth/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Outcome-aware Policy Recommendations

This file does outcome aware clustering followed by cluster-wise regression to get the policy recommendations.

# Select Inputs and Outputs

We only use one output at a time. Here are the list of outputs we can use.

In [2]:
chosen_output = 'crop_sales'
# Other options include:
# 'crop_sales_growth',
# 'expenditure',
# 'food_expenditure_diversification',
# 'has_medical_assistance',
# 'no_food_deficiency',
# 'productivity',
# 'productivity_growth',
# 'children_education'

# Missingness

In [9]:
imp_df = read_csv('../data/ethiopia_v23_raw.csv')
imp_cols = imp_df.columns.values
y = 2015

# Utility function for getting variables for each of the 3 years: 2011,13,15.
def for_year(var, year):
    return var + '___ethiopia_' + str(year)

y_reg = re.compile('.*___.*' + str(y) + '$')
y_cols = list(filter(y_reg.search, imp_cols))
imp_df = imp_df[y_cols]
output = for_year('crop_sales___output', y)
imp_df = imp_df.loc[imp_df[output].dropna().index]
missing = (len(imp_df.index) - imp_df.count())/len(imp_df.index)*100.0
missing[missing > 1]

children_education___output___ethiopia_2015                  16.456970
food_expenditure_diversification___output___ethiopia_2015     1.962758
owns_land_certificate___policy___ethiopia_2015                1.157524
quantity_of_improved_seeds_used___policy___ethiopia_2015      4.579768
dtype: float64

# Feature choice for regression based on correl and vif

In [11]:
imp_df = read_csv('../data/ethiopia_v23_raw.csv')
imp_cols = imp_df.columns.values


def select_all_year_feats():
    imp_feats = set([])
#     for y in [2013, 2015]:
    for y in [2011, 2013, 2015]:
        y_reg = re.compile('.*___policy.*' + str(y) + '$')
        y_cols = set(filter(y_reg.search, imp_cols))
        y_cols = set([t.replace('___ethiopia_' + str(y), '') for t in y_cols])
        if len(imp_feats) == 0:
            imp_feats = y_cols
        else:
            imp_feats = imp_feats.intersection(y_cols)
    return imp_feats

def select_corr_feats(thres = 0.1, only_policy=False):
    ccs = pd.read_csv('../data/cross_correls_ethiopia_2015_v22.csv')
    output = for_year('crop_sales___output', 2015)
    ccs[output] = ccs[output].apply(lambda x: abs(x))
    ccs = ccs[ccs[output] > thres]
    imp_feats = ccs['Unnamed: 0']
    non_raw_reg = re.compile('^((?!lives_in|longitude|latitude).)*$')
    imp_feats = list(filter(non_raw_reg.search, imp_feats))
    if only_policy:
        y_reg = re.compile('.*___policy.*$')
        imp_feats = list(filter(y_reg.search, imp_feats))
    return imp_feats

def normalize():
    raw_df = read_csv('../data/ethiopia_v23_raw.csv')
    output = for_year('crop_sales___output', 2015)
    raw_df = raw_df.loc[raw_df[output].dropna().index]
    raw_imp = Imputer(strategy="mean")
    raw_completed = raw_imp.fit_transform(raw_df)
    raw_cols = raw_df.columns.values
    raw_df = pd.DataFrame(raw_completed,columns=raw_cols)
    return raw_df

from statsmodels.stats.outliers_influence import variance_inflation_factor as vif_score

def spearman():
    collinear = raw_df[feats].corr('spearman')

    cols = collinear.columns
    for c in collinear.columns:
        for v in zip(cols, collinear[c]):
            if v[1] > 0.4 and v[1] < 0.5 and v[1] != 1:
                print (c, v[0], v[1])
    collinear.to_csv('../results/output_collinear.csv')
    
def vif(feats, raw_df, thres=5, debug=False):    
    while True:
        policy = raw_df[feats].as_matrix()
        max_vif = 0
        max_vif_feat = None
        for i, f in enumerate(feats):
            if max_vif < vif_score(policy, i):
                max_vif = vif_score(policy, i)
                max_vif_feat = f
        if max_vif < thres:
            break
        feat_set = set(feats)
        feat_set.remove(max_vif_feat)
        if debug:
            print ('Removed: {0} : {1}'.format(max_vif_feat, max_vif))
        feats = list(feat_set)
    if debug:
        print ('\nFinal chosen features \n')
        for i, f in enumerate(feats):
            print (f, vif_score(policy, i))
    
    return feats

raw_df = normalize()
vifs = []

# Grid search to find correlation and VIF score thresholds.
def grid_search():
    c_thresholds = [0.05, 0.07, 0.1, 0.12, 0.14,  0.15, 0.16, 0.17, 0.2,0.25,0.3]
    v_thresholds = [2,3,4,5]
    for thres in v_thresholds:
        imp_feats = select_corr_feats(0.1)
        v = vif(imp_feats, raw_df, thres)
        vifs += [len(v)]

# Chosen values of thresholds after grid search.    
c = 0.1
v = 1.5
imp_feats = select_corr_feats(c)
all_relevant = vif(imp_feats, raw_df, v, True)

imp_feats = select_corr_feats(c, True)
policy_relevant = vif(imp_feats, raw_df, v, True)


Removed: average_temperature___ethiopia_2015 : 18.472989798562818
Removed: elevation___ethiopia_2015 : 13.182135429557304
Removed: household_size___ethiopia_2015 : 7.279737966100593
Removed: average_precipitation___ethiopia_2015 : 5.572895977396103
Removed: household_head_is_male___ethiopia_2015 : 3.6755065060487286
Removed: distance_to_population_center___ethiopia_2015 : 3.0143175152781425
Removed: number_of_plough_owned___policy___ethiopia_2015 : 2.2687246376740005
Removed: number_of_oxen_owned___policy___ethiopia_2015 : 2.0068706529980638
Removed: uses_extension_program___policy___ethiopia_2015 : 1.7790754637073296
Removed: land_surface___ethiopia_2015 : 1.5292427289340589

Final chosen features 

prevent_damage___policy___ethiopia_2015 1.19743605819
has_saved___policy___ethiopia_2015 1.13110452009
number_of_droughts___ethiopia_2015 1.07379545835
uses_irrigation___policy___ethiopia_2015 1.06132163347
quantity_of_chemical_fertilizers_used___policy___ethiopia_2015 1.03744894401
distan

# Functions used for outcome aware clustering

In [19]:
# Used for segmentation based on output directly. This was used for baseline for outcome-aware clustering.
# Currently not used.
def get_classes(output_var, pred):
    max_bins = 3
    _, boundaries = np.histogram(output_var, bins=max_bins)
    classes = np.digitize(pred, bins=boundaries)
    return classes, max_bins

# Writes policy recommendations in results files after doing cluster specific outcome-aware clustering. 
def run_regressions(in_name, out_dir, non_policy_inputs, segment_variables, inputs, output, year):
    # args:
    # in_name: input file name
    # out_dir: output directory where the results (coefficients, averages per cluster, etc) are written.
    # non_policy_inputs: list of all non-actionable inputs
    # segment_variables: list of variables on which the outcome-aware clustering is done
    # inputs: list of all actionable inputs
    # output: the output variable
    # year: the year for which the policy recommendations are generated
    
    # table for storing the average regression error for population vs cluster specific clusters.
    global table
    # table for storing coefficients of the regressions
    global coef_table
    # table for storing average, std deviation, std error values of the inputs for each of the clusters
    global avg_table
    # Map of coefficients, this is an intermediate map which gets then translated to coef_table
    global coef_map
    table = pd.DataFrame()
    avg_table = pd.DataFrame()
    coef_map = {}
    coef_table = pd.DataFrame()
    
    try:
        os.mkdir(out_dir)
    except:
        print("Dir exists")
    
    # pick which regression algos to use
    regression_algorithms = ()
   
    # Read raw and normalized values
    df = read_csv(in_name)
    raw_df = read_csv(in_name.replace('normed', 'raw'))
    
    # drop rows with unobserved income
    df = df.loc[df[output].dropna().index]
    raw_df = raw_df.loc[df[output].dropna().index]
    df['nid']= df.index.tolist()

    # select % of data in test set
    test_split = 0.2
    
    # perform matrix completion. completed is returned as a np array
    # we've discussed not using it, but I left it in because I wasn't able to
    # fit the StandardScaler with a DataFrame that contained NaN's
    imp = Imputer(strategy="mean")
    completed = imp.fit_transform(df)

    # reconstruct dataframes for normalized and raw matrix with completed matrix
    cols = df.columns.values
    mat = pd.DataFrame(completed,columns=cols)
    raw_df['nid']= raw_df.index.tolist()
    raw_imp = Imputer(strategy="mean")
    raw_completed = raw_imp.fit_transform(raw_df)
    raw_cols = raw_df.columns.values
    raw_df = pd.DataFrame(raw_completed,columns=raw_cols)
    
    # Add productivity as a synthetic output based on crop sales per land surface.
    raw_df['productivity'] = raw_df[for_year('crop_sales___output', 2015)]/raw_df[for_year('land_surface', 2015)]
    raw_df['productivity'] = raw_df['productivity'].apply(lambda x: 0 if x == np.inf else x)

    # Re-scale completed data by doing z-scoring per feature.
    mat_scaled = StandardScaler()
    mat_scaled.fit(mat)
    mat_sc = mat_scaled.transform(mat)
    mat = pd.DataFrame(mat_sc, columns=mat.columns)
    
    # Optionally add aggregated input variables which could be used. This is however not used in the regression variables.
    mat[for_year('animals___policy', year)] = mat[for_year('number_of_oxen_owned___policy', year)] + mat[for_year('number_of_plough_owned___policy', year)] + mat[for_year('number_of_axe_owned___policy', year)] + mat[for_year('number_of_pick_axe_owned___policy', year)] + mat[for_year('number_of_sickle_owned___policy', year)]
    mat[for_year('tools___policy', year)] = mat[for_year('number_of_axe_owned___policy', year)] + mat[for_year('number_of_pick_axe_owned___policy', year)] + mat[for_year('number_of_sickle_owned___policy', year)]
    inputs += [for_year('animals___policy', year), for_year('tools___policy', year)]

    y = mat[output]
    x = mat[inputs]
    
    # When using Lasso, this function returns the best lambda using glmnet.
    def update_best_lambda(x_scaled):
        copy_y = np.array(y, dtype=np.float64)
        print (copy_y)
        fit = cvglmnet(x = x_scaled.copy(), y = copy_y)
        print ('Best alpha={0}'.format(fit['lambda_min']))
        return fit['lambda_min']
    
    # Split test/train
    indices = range(len(mat))
    x_train, x_test, y_train, y_test, ind_train, ind_test = \
        train_test_split(x, y, indices, test_size=test_split, random_state=42)
    
    # Returns the input rows as per the split per testing/training.
    def get_train_test(input_vars):
        x = mat[input_vars].copy()
        x_scaled = StandardScaler()
        x_scaled.fit(x)
        x_sc = x_scaled.transform(x)
        # reconstruct DataFrame
        x = pd.DataFrame(x_sc, columns=x.columns)
        training_x = x.iloc[ind_train, :]
        testing_x = x.iloc[ind_test, :]
        return x_sc, training_x, testing_x
    
    # This is to convert baseline segments into one-hot encoding. 
    # Currently not used.
    def digitize(output_var, pred):
        from sklearn.preprocessing import label_binarize
        classes, max_bins = get_classes(output_var, pred)
        b_classes = label_binarize(classes, range(max_bins))
        return b_classes
    
    # Do regressions without clustering, this gives the population level recommendations.
    def calc_unsegmented(baseline, x_train, x_test): 
        global table
        global coef_map

        # Convert test dataframe to list to pass to regression equation.
        keys_list = []
        y_list = []
        for k, v in y_test.iteritems():
            keys_list.append(k)
            y_list.append(v)

        # run regressions on population level using OLS.
        name = 'OLS'
        # If you want to run Lasso, change this to True.
        run_regularized = False
        model = sm.OLS(y_train, x_train)
        if run_regularized:
            fit = model.fit_regularized(alpha=0.006, refit=True)
        else:
            fit = model.fit()
        print("Confidence intervals are {0}".format(fit.conf_int()))
        y_pred = fit.predict(x_test)

        # add MSE and AUC to table
        try:
            test_c = digitize(y, y_test)
            pred_c = digitize(y, y_pred)
            auc_c = roc_auc_score(test_c, pred_c, average='macro')
        except ValueError:
            auc_c = 0.5
        mse = mean_squared_error(y_test,y_pred)
        scaled_mse = (mse/np.std(y))
        new_row = pd.DataFrame({'model': name, 'segment': '', 'input': baseline, 'scaled_mse': scaled_mse, 'mse': mse, 'roc_auc': auc_c, 'clustered': False}, index=[0])
        table = table.append(new_row, ignore_index=True)

        # add coefficients to map
        coef_map[name + '_' + baseline] = fit.params
        lower_bounds = []
        upper_bounds = []
        ci = fit.conf_int()
        if run_regularized:
            for ci in fit.conf_int():
                lower_bounds += [ci[0]]
                upper_bounds += [ci[1]]
        else:
            for ci_row in ci.iterrows():
                lower_bounds += [ci_row[1][0]]
                upper_bounds += [ci_row[1][1]]
        coef_map[name + '_' + baseline + '_lower_bound'] = lower_bounds
        coef_map[name + '_' + baseline + '_upper_bound'] = upper_bounds
    
    def calc_segmented(segment_variables, baseline, x_train, x_test):
        global table
        global coef_map
        global avg_table

        # list of variables to segment on.
        segment_vars = list(segment_variables.keys())
        
        # Choosing iteratively the features to do outcome aware clustering on.
        # Returns the silhouette coefficient and the kmeans data object with labels in them.
        def iterative_clustering(sc_thres, corr_thres, alpha):
            max_sc = 0
            best_kmeans = None

            sc_incr = 1e-8
            sc_thres = -1e-1
            C = set([])
            avail = set(segment_variables.keys())
            prev_sc = 0
            min_k = 4
            corr_thres = 0.05
            while sc_incr >= sc_thres:
                u_f = 0
                best_f = None
                sc_best = 0
                for f in avail:
                    if segment_variables[f] < corr_thres:
                        continue
                    seg_data = mat[list(avail) + [f]]
                    kmeans = KMeans(n_clusters=min_k).fit(seg_data)
                    labels = kmeans.labels_
                    sc = silhouette_score(seg_data, labels)
                    sc_delta = sc - prev_sc
                    if u_f < (segment_variables[f] + (alpha)*sc_delta):
                        u_f = (segment_variables[f] + (alpha)*sc_delta)
                        best_f = f
                        sc_best = sc
                if best_f == None:
                    break
                avail.remove(best_f)
                C.add(best_f)
                sc_incr = sc - prev_sc
                prev_sc = sc
            
            seg_data = mat[list(C)]
            kmeans = KMeans(n_clusters=min_k).fit(seg_data)
            labels = kmeans.labels_
            sc = silhouette_score(seg_data, labels)
            return sc, kmeans

        
        def add_clusters():
            # elbow method
            sse = []
           
            # Multiply by correlation coefficient to make weigh feature distances as per outcome importance.
            seg_data = mat[segment_vars]
            for seg_var in segment_vars:
                seg_data[seg_var] = seg_data[seg_var].apply(lambda x: x*segment_variables[seg_var])
    
            ### Plot elbow. Currently not run. Do this once before fixing k.
            def plot_elbow():
                for k in range(1,9):
                    kmeans = KMeans(n_clusters=k).fit(seg_data)
                    labels = kmeans.labels_
                    sse.append(sum(np.min(cdist(seg_data, kmeans.cluster_centers_, 'euclidean'), axis=1)) / seg_data.shape[0])

                plt.clf()
                plt.plot(range(1,9), sse)
                plt.xlabel('k')
                plt.ylabel('Sum of squared error')
                plt.savefig(os.path.join(out_dir, 'elbow.png'))
                print(sse)
                min_k = sse.index(min(sse))
                print (min_k)

            ### Parameter search for iterative clustering. Use this while doing search over parameters
            def best_kmeans_params():
                max_sc = -1
                best_kmeans = None
                for sc_thres in [-0.1, -0.05, -0.01, -0.001, -1e-3, 0, 1e-3, 1e-2]:
                    for corr_thres in [0.05, 0.06, 0.08, 0.1, 0.12, 0.15]:
                        for alpha in [1e-4,1e-3,1e-2,0.1,1,10,100,1000,10000]:
                            sc, kmeans = iterative_clustering(sc_thres, corr_thres, alpha)
                            if sc > max_sc:
                                max_sc = sc
                                best_kmeans = kmeans
                                print ('Best found: {0}, {1}, {2}'.format(sc_thres, corr_thres, alpha))
            
                return best_kmeans
    
            min_k = 4
            # After features are chosen, simply do kmeans. Uncomment below line to redo the feature choice.
            # kmeans = best_kmeans_params()
            
            kmeans = KMeans(n_clusters=min_k).fit(seg_data)
            labels = kmeans.labels_
            mat['cluster'] = labels
            
            # Record average segment variable values per cluster and write to file.
            means = []
            for i in np.unique(labels):
                sel_mat = mat[mat['cluster'] == i]
                raw_clus = raw_df.loc[sel_mat.index]
                values = raw_clus[output].as_matrix()
                weights = raw_clus[for_year('weight', year)].as_matrix()
                average = np.average(values, weights=weights)
                means.append(average)
            sorted_ids = [i[0] for i in sorted(enumerate(means), key=lambda x:x[1])]
            mat['cluster'] = mat['cluster'].apply(lambda x: sorted_ids.index(x))

            # Add location features to plot the clusters on the map.
            select_variables = [output, 'latitude___ethiopia_2015', 'longitude___ethiopia_2015', 'nid'] + segment_vars
            select_variables = [t.replace('norm', 'raw') for t in select_variables]
            all_output = pd.concat([mat['cluster'], raw_df[list(select_variables)]], 1)
            all_output.to_csv(os.path.join(out_dir,'clus_' + baseline + '_' + output + '.csv'))
            return min_k
            
        
        # Add segments. Used for heuristic output based segmentation for baselines. Not used anymore.
        def add_segments():
            median_segments = {}
            for seg_var in segment_vars:
                #binary
                if len(np.unique(mat[seg_var])) <= 3:
                    median_segments[seg_var] = 0
                else:
                    median_segments[seg_var] = np.median(mat[seg_var])
            mat['cluster'] = 0
            for seg_var in segment_vars:
                mat['cluster'] = 2*mat['cluster'] + [int(x) for x in mat[seg_var] > median_segments[seg_var]]
            return int(math.pow(2, len(segment_vars)))

        # Add location specific segments. Used as baseline. Not used anymore.
        def add_location_segments():
            locations = ['afar', 'amhara', 'benishangul_gumuz', 'dire_dawa', 'gambella', 'harari', 'oromiya', 'snnp', 'somalie', 'tigray']
            i = 0
            mat['cluster'] = 0
            for l in sorted(locations):
                loc_feature = 'lives_in_' + l + '___ethiopia_' + str(year)
                loc_val = mat[loc_feature].apply(lambda x: 0 if x < 0 else 1)
                mat['cluster'] = mat['cluster'] + (i*loc_val)
                i += 1
            return len(locations) + 1
            
        # Base function which aggregates cluster based regression procedures, given the clusters.
        def _run(max_clusters, method_name):
            global table
            global avg_table
            global coef_map
            # reg_clus keeps predictions from clustered regressions along with keys
            reg_clus = dict()
            name = 'OLS'
            reg_clus[name] = {}

            # need new dataframes with only training and test rows.
            # we use this when looping through clusters
            train_mat = mat.loc[ind_train]
            test_mat = mat.loc[ind_test]
            train_size = len(train_mat)
            series = {}
            series[output] = []
            for seg in segment_vars:
                series[seg] = []
            series = pd.DataFrame()
            row = {}
            raw_cols = raw_df.columns.values
            
            # Keep only normed values.
            raw_reg = re.compile('^((?!norm).)*2015$')
            avg_variables = list(filter(raw_reg.search, raw_cols))

            # Iterate through each of the clusters and
            # 1. Store the average input variables per cluster
            # 2. Do Regressions per cluster
            for i in range(max_clusters):
                train_clus = x_train.loc[train_mat['cluster'] == i]
                train_y = y_train.loc[train_mat['cluster'] == i]
                test_clus = x_test.loc[test_mat['cluster'] == i]
                test_y = y_test.loc[test_mat['cluster'] == i]
                sel_mat = mat[mat['cluster'] == i]
                raw_clus = raw_df.loc[sel_mat.index]

                # Store average input variables.
                for seg in avg_variables:
                    values = raw_clus[seg].as_matrix()
                    weights = raw_clus[for_year('weight', year)].as_matrix()
                    average = np.average(values, weights=weights)
                    row['mean_' + seg] = average
                    row['avg_' + seg] = np.mean(values)
                    variance = np.average((values-average)**2, weights=weights)
                    row['stddev_' + seg] = math.sqrt(variance)
                    row['stderr_' + seg] = math.sqrt(variance)/math.sqrt(len(values))
                    row['25ile_' + seg] = np.percentile(values, 25)
                    row['75ile_' + seg] = np.percentile(values, 75)

                row['index'] = i
                row['size'] = len(raw_clus)
                new_row = pd.DataFrame(row, index=[0])
                series = series.append(new_row, ignore_index=True)
                avg_table = avg_table.append(new_row, ignore_index=True)
                cluster_percent = (len(train_clus)*100.0)/train_size
                if train_clus.empty or test_clus.empty or cluster_percent < 5:
                    print("Cluster size too small, retry!")
                    continue

                keys_list = []
                y_list = []
                for k, v in test_y.iteritems():
                    keys_list.append(k)
                    y_list.append(v)

                model = sm.OLS(train_y, train_clus)
                run_regularized = False
                if run_regularized:
                    fit = model.fit_regularized(alpha=0.00617565, refit=True)
                else:
                    fit = model.fit()
                print("Confidence intervals are {0}".format(fit.conf_int()))
                y_pred = fit.predict(test_clus)

                for a, b in enumerate(y_pred):
                    t = reg_clus[name]
                    t[keys_list[a]] = b

                coef_map[name + '_' + method_name + '_' + ','.join(segment_vars) + '_' + str(i)] = fit.params
                lower_bounds = []
                upper_bounds = []
                ci = fit.conf_int()
                if run_regularized:
                    for ci in fit.conf_int():
                        lower_bounds += [ci[0]]
                        upper_bounds += [ci[1]]
                else:
                    for ci_row in ci.iterrows():
                        lower_bounds += [ci_row[1][0]]
                        upper_bounds += [ci_row[1][1]]
                coef_map[name + '_' + method_name + '_' + str(i) + '_lower_bound'] = lower_bounds
                coef_map[name + '_' + method_name + '_' + str(i) + '_upper_bound'] = upper_bounds

            # plot sorted correlation between average inputs across clusters.
            sorted_series = series.sort_values(['mean_' + output])
            raw_segment_vars = avg_variables
            for seg in raw_segment_vars:
                plt.clf()
                plt.plot(sorted_series['mean_' + output].as_matrix(), sorted_series['mean_'+seg].as_matrix(), marker='o')
                plt.xlabel('Average ' + output.replace('___output___ethiopia_2015', '') + ' output')
                plt.ylabel('Average ' + seg.replace('___policy___ethiopia_' +str(year), '').replace('_',' '))
                plt.savefig(os.path.join(out_dir, 'plot_' + seg + '_' + output + '.pdf'))
                
            # add mse's to table
            keys = sorted(y_test.keys())
            sort_t = []
            sort_p = []
            for key in keys:
                if key not in y_test or key not in reg_clus[name]:
                    continue
                sort_t.append(reg_clus[name][key])
                sort_p.append(y_test[key])

            try:
                test_c = digitize(y, sort_t)
                pred_c = digitize(y, sort_p)
                auc_c = roc_auc_score(test_c, pred_c, average='macro')
            except ValueError:
                auc_c = 0.5
            mse = mean_squared_error(sort_t,sort_p)
            scaled_mse = (mse/np.std(y))
            new_row = pd.DataFrame({'model': name, 'segment': ','.join(segment_vars), 'input': baseline, 'scaled_mse': scaled_mse, 'mse': mse, 'roc_auc': auc_c, 'clustered': True, 'method': method_name}, index=[0])
            table = table.append(new_row, ignore_index=True)
                
    
        ##### Run grouped regressions
        if (len(segment_vars) > 1):
            _run(add_clusters(), 'clustered')
        
        # Uncomment if you want to get baseline if segmentation is done based on raw output value/location.
        #_run(add_location_segments(), 'segmented')
        #_run(add_segments(), 'segmented')
        

    # Runs outcome aware clustering policy recommendations along with population level baseline given the set
    # of input variables to be considered for regression.
    def run_with_inputs(input_vars, name):
        # map to be used in tracking coefficients
        global coef_map
        global coef_table
        global x_train
        global x_test
        coef_map = {}
        x_scaled, x_train, x_test = get_train_test(input_vars)
        
        # Use this lambda when running Lasso. Currently unused as we use OLS.
        update_best_lambda(x_scaled)
        calc_unsegmented(name, x_train, x_test)
        calc_segmented(segment_variables, name, x_train, x_test)
        
        # Convert coef_map to coef_table with column headers.
        for k,v in sorted(coef_map.items()):
            kvp = dict()
            kvp['model'] = k
            kvp['inputs'] = name

            for val,invar in zip(v,input_vars):
                kvp[invar] = val

            new_row = pd.DataFrame(kvp, index=[0])
            coef_table = coef_table.append(new_row, ignore_index=True)
    
    # Use the relevant variables generated above for regression here.
    # TODO(ananth): Plumb it through within the function.
    global all_relevant, policy_relevant
    print ('Relevant variables')
    print(all_relevant, policy_relevant)
    run_with_inputs(all_relevant, 'Highly correlated all')
    run_with_inputs(policy_relevant, 'Highly correlated policy')
    
    # Baseline using all possible inputs without removing collinear variables. Uncomment for comparison.
    # input_vars = inputs + non_policy_inputs
    # run_with_inputs(input_vars, 'All variables')
    
    # save coefficient and output tables
    coef_table.to_csv(os.path.join(out_dir,'coef_' + output + '.csv'))
    table.to_csv(os.path.join(out_dir,output + '.csv'))
    avg_table.to_csv(os.path.join(out_dir,output + '_avg' + '.csv'))
    

# Utility function to complete the missing matrix features.
def complete(df):
    imp = Imputer(strategy="mean")
    completed = imp.fit_transform(df)
    cols = df.columns.values
    mat = pd.DataFrame(completed,columns=cols)
    mat_scaled = StandardScaler()
    mat_scaled.fit(mat)
    mat_sc = mat_scaled.transform(mat)
    mat = pd.DataFrame(mat_sc, columns=mat.columns)
    return mat


# Utility function to do clustering. This is currently used to do multiple year clusters comparison.
def get_clusters(mat, output, segment_vars, out_dir):
    segment_vars = list(segment_vars.keys())
    seg_data = mat[segment_vars]
    min_k = 4
    kmeans = KMeans(n_clusters=min_k).fit(seg_data)
    labels = kmeans.labels_
    means = []
    for i in np.unique(labels):
        df_clus = mat.loc[labels == i]
        means.append(np.mean(df_clus[output].as_matrix()))
    sorted_ids = [i[0] for i in sorted(enumerate(means), key=lambda x:x[1])]
    mat['segment_'+ output] = labels
    mat['segment_'+ output] = mat['segment_'+ output].apply(lambda x: sorted_ids.index(x))
    select_variables = [output, 'latitude___ethiopia_2015', 'longitude___ethiopia_2015'] + segment_vars
    select_variables = [t.replace('norm', 'raw') for t in select_variables]
    all_output = pd.concat([mat['segment_' + output], raw_df[select_variables]], 1)
    all_output.to_csv(os.path.join(out_dir,'clus_' + '_' + output + '.csv'))
    return mat

# Run the above functions for Ethiopia dataset

In [28]:
import re

filename = '../data/ethiopia_v23_normed.csv'
df = pd.read_csv(filename)
# If true, we will generate only clusters across multiple years. Use this mode, when policy recommendations
# for a given year is already computed and the lifts need to be computed.
generate_multiple_years_clusters = True

# Extract actionable, non-actionable variables and the output from the full list of columns.
def get_vars(year, base_output):
    base_suffix = '___ethiopia_{0}'.format(year)
    year_vars = df.filter(regex='.*{0}'.format(base_suffix)).columns.values
    out_reg = re.compile('.*' + base_output + '___output.*$')
    outputs = list(filter(out_reg.search, year_vars))
    policy_reg = re.compile('(.*___policy.*)$')
    policy_inputs = list(filter(policy_reg.search, year_vars))
    non_policy_reg = re.compile('^((?!policy|output|weight).)*$')
    non_policy_inputs = list(filter(non_policy_reg.search, year_vars))
    
    return outputs, policy_inputs, non_policy_inputs

if generate_multiple_years_clusters:
    years = [2011, 2013, 2015]
else:
    years = [2015]
    
for base_output in [chosen_output]:
    ccs = pd.read_csv('../data/cross_correls_ethiopia_2015_v22.csv')
    output = base_output + '___output___ethiopia_2015'
    ccs[output] = ccs[output].apply(lambda x: abs(x))
    
    # Set of 8 segmenting variables chosen based on iterative feature choosing mentioned above.
    # Choose correlation values from selected variables
    select = ccs['Unnamed: 0'].str.contains('^.*(oxen|extension|hired|chemical|plough|land_surface|household_size)') # gender, has_saved
    ccs = ccs[select]
    ccs = ccs.sort_values(output, ascending=False)
    num_vars=8
    
    best_vars = ccs[['Unnamed: 0', output]][:num_vars].as_matrix()
    segment_variables = {}
    seg_vars = []
    for i in best_vars:
        name = i[0]
        # Location and distance variables are not well defined, ignore.
        if 'lives' in name or 'distance' in name:
            continue
        segment_variables[name] = abs(i[1])

    print(segment_variables)

    raw_df = df.copy()
    df = complete(df)
    for year in years:
        base_suffix = '___ethiopia_{0}'.format(year)
        year_segment_variables = {}
        for k, v in segment_variables.items():
            year_segment_variables[k.replace('2015', str(year))] = v
        outputs, policy_inputs, non_policy_inputs = get_vars(year, base_output)
        print(outputs)
        raw_df = normalize()
        for output in outputs:
            if generate_multiple_years_clusters:
                df = get_clusters(df, output, year_segment_variables, out_dir='../results/ethiopia_crop_sales')
            else:
                for c in [0.05]:
                    for v in [1.5]:
                        out_dir = '../results/grid_' + base_output + '/{0}/{1}'.format(c, v)
                        try:
                            os.makedirs(out_dir)
                        except:
                            print('File exists: ' + out_dir)
                        global all_relevant, policy_relevant
                        imp_feats = select_corr_feats(c)
                        all_relevant = vif(imp_feats, raw_df, v)
                        run_regressions(in_name=filename, out_dir=out_dir, non_policy_inputs=non_policy_inputs, segment_variables=segment_variables, inputs=policy_inputs, output=output, year=year)

                        imp_feats = select_corr_feats(c, True)
                        policy_relevant = vif(imp_feats, raw_df, v)
                        run_regressions(in_name=filename, out_dir=out_dir, non_policy_inputs=non_policy_inputs, segment_variables=segment_variables, inputs=policy_inputs, output=output, year=year)

/home/ananth/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


{'number_of_hired_workers___policy___ethiopia_2015': 0.273808267736554, 'land_surface___ethiopia_2015': 0.272516052599146, 'quantity_of_chemical_fertilizers_used___policy___ethiopia_2015': 0.160090553373268, 'number_of_oxen_owned___policy___ethiopia_2015': 0.157053494140115, 'household_size___ethiopia_2015': 0.14258227392332698, 'number_of_plough_owned___policy___ethiopia_2015': 0.14223988258113301, 'uses_extension_program___policy___ethiopia_2015': 0.12981246190952803}
['crop_sales___output___ethiopia_2011']
['crop_sales___output___ethiopia_2013']
['crop_sales___output___ethiopia_2015']


# Evidence of movement across years

In [29]:
# Copy dataframe to a new variable once which will not be altered.
df_all = df
assert generate_multiple_years_clusters==True, "Ensure that df has clusters of all years populated by setting this to True, first"

In [30]:
df = df_all

imp_feats = select_all_year_feats()

series = pd.DataFrame()
output = 'segment_crop_sales___output'
raw_output = 'crop_sales___output'

# Compute conditional lifts in output when there is increase in each of the features across years.
for imp_feat in imp_feats:
    for [y1, y2] in [['2011', '2013'], ['2013', '2015']]:
        z1 = for_year(output, y1)
        z2 = for_year(output, y2)
        r1 = for_year(raw_output, y1)
        r2 = for_year(raw_output, y2)
        f1 = for_year(imp_feat,y1)
        f2 = for_year(imp_feat,y2)
        df[output + '_change' + y1] = (df[z1]!=df[z2])
        df[output + '_increase' + y1] = (df[z1]<df[z2])
        df[output + '_decrease' + y1] = (df[z1]>df[z2])
        df[imp_feat+'_increase'+y1] = (imp_df[f2]-imp_df[f1])
        df[output + '_change_value' + y1] = (imp_df[r2]-imp_df[r1])

    # Percentile thresholds if using one to take only significant changes into account.
    # per_thresholds = [0, 5,10,15,20,25,30,35,40,45,50,55,60,70,75,80,85,90,95]
    
    # Iterate through all clusters to see if different variables have different lifts.
    for per in [0]:
        for seg in range(4):
            exp_y_i = []
            exp_y = []
            exp = []
            exp_i = []
            avg_y_i = []
            avg_y = []
            std_y_i = []
            std_y = []

            for [y1, y2] in [['2011', '2013'], ['2013', '2015']]:
                dec_y = []
                num_dec_y = []
                dec_base_y = []
                year = y1
                weight = for_year('weight', year)
                seg_y = for_year(output, year)
                df_seg = df[df[seg_y]==seg]
                df_seg = df_seg.loc[df_seg[output + '_change_value' + year].dropna().index]
                high_df = df_seg
                std_dev = np.std(high_df[imp_feat + '_increase'+year])
                feat_mean = np.mean(high_df[imp_feat + '_increase'+year])
                def get_thresholds(v):
                    if v < -std_dev:
                        return 0
                    elif v < 0:
                        return 1
                    elif v > std_dev:
                        return 3
                    elif v >= 0:
                        return 2

                median = np.median(high_df[imp_feat + '_increase'+year].dropna())
                print (imp_feat, median, year)
                def get_simple_thresholds(v):
                    if v < median:
                        return 0
                    elif v == median:
                        return 1
                    elif v > median:
                        return 2
                    else:
                        return -1

                # Compute sub-populations' lift based on increase, no-change or decrease in the input feature.
                high_df[imp_feat + '_decile'+year] = high_df[imp_feat + '_increase'+year].apply(lambda x: get_simple_thresholds(x))
                clus_avg = np.mean(df_seg[output + '_change_value' + year])
                dec_base_y = clus_avg
                for t in range(3):
                    num_dec_y.append(len(high_df[high_df[imp_feat + '_decile'+year]==t]))
                    dec_y.append(np.mean(high_df[high_df[imp_feat + '_decile'+year]==t][output + '_change_value' + year])/clus_avg)
                
                exp_y_i.append(len(high_df[(high_df[imp_feat + '_increase'+year]>0)]))
                exp_y.append(len(high_df))
                exp = len(df_seg)
                exp_i.append(len(df_seg[df_seg[imp_feat + '_increase'+year]>0]))
                avg_y_i.append(np.mean(high_df[(high_df[imp_feat + '_increase'+year]>0)][output + '_change_value' + year]))
                avg_y.append(np.mean(high_df[output + '_change_value' + year].dropna()))
                std_y_i.append(np.std(high_df[(high_df[imp_feat + '_increase'+year]>0)][output + '_change_value' + year]))
                std_y.append(np.std(high_df[output + '_change_value' + year].dropna()))
                avg = np.median(df[output + '_change_value' + year].dropna())
                row = {}
                row['threshold'] = per
                row['input'] = imp_feat
                row['from'] = year
                row['to'] = y2
                row['cluster #'] = seg
                row['num_hh_in_cluster'] = exp
                row['num_hh_in_bins'] = str(num_dec_y)
                row['binned_conditional_lift_ratios'] = str(dec_y) 
                row['average_change_in_output'] = dec_base_y
                new_row = pd.DataFrame(row, index=[0])
                series = series.append(new_row, ignore_index=True)

series.to_csv('../results/threshold-lift-simple.csv')


number_of_pick_axe_owned___policy 0.0 2011
number_of_pick_axe_owned___policy 0.0 2013
number_of_pick_axe_owned___policy 0.0 2011
number_of_pick_axe_owned___policy 0.0 2013
number_of_pick_axe_owned___policy 0.0 2011
number_of_pick_axe_owned___policy 0.0 2013
number_of_pick_axe_owned___policy 0.0 2011
number_of_pick_axe_owned___policy 0.0 2013
number_of_sickle_owned___policy 0.0 2011
number_of_sickle_owned___policy 0.0 2013
number_of_sickle_owned___policy 0.0 2011
number_of_sickle_owned___policy 0.0 2013
number_of_sickle_owned___policy 0.0 2011
number_of_sickle_owned___policy 0.0 2013
number_of_sickle_owned___policy 0.0 2011
number_of_sickle_owned___policy 0.0 2013
number_of_hired_workers___policy 0.0 2011
number_of_hired_workers___policy 0.0 2013
number_of_hired_workers___policy 0.0 2011
number_of_hired_workers___policy 0.0 2013
number_of_hired_workers___policy 0.0 2011
number_of_hired_workers___policy 0.0 2013
number_of_hired_workers___policy -9.0 2011
number_of_hired_workers___policy 